# 研究ディレクトリ作成用テンプレート

これはDMP情報に基づき、研究ディレクトリを作成するためのノートブックです。  
システムによってパラメタライズされるため、手動での実行は非推奨です。

## ディレクトリ作成情報の変数を用意する
papermillでワークフロー生成機能がdmp.jsonから取得した情報にオーバーライドされる

In [ ]:
DMP_PATH      = "" # DMP_PATH is override by papermill
RESEARCH_PATH = "" # RESEARCH_PATH is override by papermill

# These parameters are also override by papermill
CWD = ""
SIBLING_NAME = ""
DEFAULT_BRANCH = ""
HTTP_HOST = ""
SSH_HOST = ""

save_path = ""
save_message = ""

In [ ]:
import os
import csv
import json
from collections import OrderedDict
import pprint

with open(DMP_PATH, 'r') as f:
#     line = [row for row in csv.reader(f)]
    jsn = json.load(f)

for research in jsn['researches']:
    os.makedirs(RESEARCH_PATH+research['title'])
    with open(RESEARCH_PATH+research['title']+"/README.md", mode='w') as f:
        f.write("# "+research['title'])
        
#     print(jsn['researches'][0]['title'])
#     pprint.pprint(jsn, width=40)

In [ ]:
import os

os.chdir(CWD)

In [ ]:
from datalad import api

api.save(path=save_path, message=save_message)

In [ ]:
%%sh
chmod 600 /home/jovyan/.ssh/id_rsa

In [ ]:
%%sh
mkdir ./tmp
git remote show origin > ./tmp/remote.txt

In [ ]:
from datalad import api

# SSHでの接続URLを生成
with open('./tmp/remote.txt', 'r') as f:
    lines = [s.strip() for s in f.readlines()]
    ssh_remote = lines[2].replace('Push  URL: '+HTTP_HOST, SSH_HOST)

try:
    api.siblings(action='add', name=SIBLING_NAME, url=ssh_remote)
except:
    pass

api.push(to=SIBLING_NAME, result_renderer='default')

In [ ]:
%%sh
rm -r ./tmp